In [2]:
pip install datasets

In [3]:
from datasets import load_dataset, Dataset
import pandas as pd


ds = load_dataset("b-mc2/sql-create-context")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
print(ds)


print(ds['train'].column_names)
print(ds['train'].features)

print(ds['train'][0])

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 78577
    })
})
['answer', 'question', 'context']
{'answer': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None)}
{'answer': 'SELECT COUNT(*) FROM head WHERE age > 56', 'question': 'How many heads of the departments are older than 56 ?', 'context': 'CREATE TABLE head (age INTEGER)'}


In [5]:
ds_small = ds['train'].train_test_split(test_size=0.1)['test']


print(ds_small)

Dataset({
    features: ['answer', 'question', 'context'],
    num_rows: 7858
})


In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import DatasetDict


ds_small_dict = DatasetDict({"train": ds_small})


tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

def preprocess_function(examples):
    inputs = ["translate English to SQL: " + ex for ex in examples['question']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['answer'], max_length=512, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_ds = ds_small_dict.map(preprocess_function, batched=True)


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [9]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    save_strategy="steps",
    save_steps=500,
    save_total_limit=3,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    data_collator=data_collator,
)


trainer.train()


trainer.save_model("./final_model")

Step,Training Loss
100,4.285700
200,3.034700
300,2.290800
400,2.069400
500,1.894600
600,1.771700
700,1.626100
800,1.661600
900,1.531000
1000,1.483500


In [15]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments


tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("./final_model")
# Tạo lại các tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    save_strategy="steps",
    save_steps=500,
    save_total_limit=3,
    learning_rate=2e-5,
    per_device_train_batch_size = 8,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    data_collator=data_collator,
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
trainer.train(resume_from_checkpoint=True)


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3087: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Pleas

Step,Training Loss


TrainOutput(global_step=2949, training_loss=0.0, metrics={'train_runtime': 0.0058, 'train_samples_per_second': 4036435.438, 'train_steps_per_second': 504939.684, 'total_flos': 210475005050880.0, 'train_loss': 0.0, 'epoch': 3.0})

In [12]:
trainer.save_model("./final_model_continued")


In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import DatasetDict, load_dataset


model = T5ForConditionalGeneration.from_pretrained("/content/final_model")


In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    save_strategy="steps",
    save_steps=500,
    save_total_limit=3,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
from transformers import Trainer, DataCollatorForSeq2Seq


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    data_collator=data_collator,
)


trainer.train()


Step,Training Loss
100,1.279000
200,1.301400
300,1.219400
400,1.244800
500,1.235600
600,1.258900
700,1.231900
800,1.179100
900,1.173200
1000,1.179300


TrainOutput(global_step=4915, training_loss=1.1026513269449663, metrics={'train_runtime': 17671.1131, 'train_samples_per_second': 2.223, 'train_steps_per_second': 0.278, 'total_flos': 348294195904512.0, 'train_loss': 1.1026513269449663, 'epoch': 5.0})

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
!cp -r /content/final_model /content/drive/MyDrive/final_model
